In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

nbdev 한글 문제 해결을 위해서는 encoding을 바꾸어야 합니다.

1. doclinks.py에서 49번째 줄 p.read_text()를 p.read_text(encoding='utf-8')으로 바꾸면 됩니다.(총 3가지가 있어서 찾아바꾸기가 좋은 것 같습니다.)
2. serve.py 파일에서도 25번째 줄에 p.read_text()를 p.read_text(encoding='utf-8)
3. setup에 있는 read 도 바꾸어야 합니다.


### gencache error 해결 

캐시가 충돌하는 문제를 해결하기 위해서 장치를 마련했습니다.
오류가 나는지 확인은 필요할 것으로 보입니다.


In [ ]:
#| export

import os
import shutil
from contextlib import contextmanager

def dispatch(app_name:str):
    try:
        from win32com import client
        app = client.gencache.EnsureDispatch(app_name)
    except AttributeError:
        # Corner case dependencies.
        import os
        import re
        import sys
        import shutil
        # Remove cache and try again.
        MODULE_LIST = [m.__name__ for m in sys.modules.values()]
        for module in MODULE_LIST:
            if re.match(r'win32com\.gen_py\..+', module):
                del sys.modules[module]
        shutil.rmtree(os.path.join(os.environ.get('LOCALAPPDATA'), 'Temp', 'gen_py'))
        from win32com import client
        app = client.gencache.EnsureDispatch(app_name)
    return app

### 필요한 패키지는 아래와 같습니다. 

In [ ]:
#| export
from win32com import client as wc
from pathlib import Path
from fastcore.basics import patch

In [ ]:
#| export 
from HwpApi.actions import _Actions, _Action
from HwpApi.functions import get_absolute_path, check_dll, get_value

ModuleNotFoundError: No module named 'HwpApi'

액션을 좀더 쓰기 편하게 하기 위해서 구성하였습니다.

App은 기존 hwpali를 wrapping 하여 보다 사용하기 쉽게 만드는 것을 목적으로 하고 있는 클래스 입니다.
한글 컨트롤과 동등한 기능입니다.
기본 메소드를 보다 쓰기 쉽게 만들고자 합니다.
앞으로 추가할 기능은 화면 크기 조정 등이 있습니다.
api 메소드를 통해 win32com의 방식으로 모두 사용할 수 있어 다양하게 사용할 수 있습니다.

기본적으로는 `App` 클래스와 `_Action`클래스가 있습니다.
`_Action` 클래스는 `App`로만 생성되어야 하기 때문에 언더바로 시작합니다.

In [ ]:
#| export
class App:
    ''' App 클래스는 한컴오피스의 한/글 프로그램과 상호작용하기 위한 인터페이스를 제공합니다.'''
    def __init__(self, api=None):
        ''' `__init__` 함수에서는 `api` 객체를 인자로 받습니다. 
        만약 `api`가 제공되지 않았을 경우, `wc.gencache.EnsureDispatch("HWPFrame.HwpObject")`를 호출하여 
        기본값으로 한/글 프로그램의 COM 객체를 생성합니다. 그리고 `self.api` 속성에 이 객체를 할당합니다. 
        `_Actions` 클래스의 객체를 생성하여 `self.actions` 속성에 할당하고, `self.set_visible()` 함수를 호출합니다.'''
        if not api:        
            api = dispatch("HWPFrame.HwpObject")
        self.api = api
        self.actions = _Actions(self)
        self.set_visible()
        check_dll()
        self.api.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")
                
    def __str__(self): return f"<Hwp App: {self.get_filepath()}>"
    __repr__ = __str__

In [ ]:
#| export
@patch
def set_visible(app:App, is_visible=True, window_i=0):
    '''`set_visible()` 함수는 한/글 프로그램의 창을 화면에 보이거나 숨기기 위해 호출됩니다. 
    `is_visible` 인자가 `True`일 경우 창이 화면에 보이고, `False`일 경우 숨깁니다. 
    `window_i` 인자는 창의 인덱스를 지정합니다.'''

    app.api.XHwpWindows.Item(window_i).Visible = is_visible

In [ ]:
#| export
@patch
def get_filepath(app:App):
    '''`get_filepath()` 함수는 현재 열려있는 한/글 문서의 경로를 반환합니다.'''
    doc = app.api.XHwpDocuments.Active_XHwpDocument
    return doc.FullName

In [ ]:
import pywintypes

try:
    app = App(app.api)
except NameError:
    app = App()
except pywintypes.com_error:
    app = App()


In [ ]:
#| export
@patch
def open(app:App, path:str):
    '''`open()` 함수는 파일 경로를 인자로 받아 해당 파일을 한/글 프로그램에서 엽니다. 
    `get_absolute_path()` 함수를 호출하여 절대 경로로 변환한 후, `api.Open()` 함수를 호출하여 파일을 엽니다. 
    열린 파일의 경로를 반환합니다.'''
    name = get_absolute_path(path)
    app.api.Open(name)
    return name

In [ ]:
app.open(r"C:\Users\freed\Downloads\일반관리비 관리 자료 작성\1. Template6_일반관리비template_0309_1730(수정).hwp")

In [ ]:
#| export
@patch
def save(app:App, path=None):
    '''`save()` 함수는 현재 열려있는 문서를 저장하거나 다른 이름으로 저장합니다. 
    `path` 인자가 주어지지 않은 경우 현재 문서를 덮어쓰기로 저장하고, 저장된 파일의 경로를 반환합니다. 
    `path` 인자가 주어진 경우, `Path` 모듈을 이용하여 파일 확장자를 추출한 후, 해당 확장자에 맞게 문서를 저장합니다. 
    저장된 파일의 경로를 반환합니다.'''

    if not path:
        app.api.Save()
        return app.get_filepath()
    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {".hwp": "HWP", ".pdf": "PDF", ".hwpx": "HWPML2X"}.get(extension)

    app.api.SaveAs(name, format_)
    return name

In [ ]:
#| export
@patch
def close(app:App):
    '''`close()` 함수는 현재 열려있는 문서를 닫습니다.'''
    app.api.Run("FileClose")

In [ ]:
#| export
@patch
def quit(app:App):
    '''`quit()` 함수는 한/글 프로그램을 종료합니다.'''
    app.api.Run("FileQuit")

In [ ]:
#| export
@patch        
def create_action(app:App, action_key:str):
    '''`create_action()` 함수는 `_Action` 클래스의 객체를 생성하여 반환합니다.'''
    return _Action(app, action_key)

In [ ]:
#| export
@patch    
def create_parameterset(app:App, action_key:str):
    '''`create_parameterset()` 함수는 특정 액션의 파라미터셋을 반환합니다.
    `_action_info` 딕셔너리에서 액션에 대한 정보를 찾아서 파라미터셋의 키 값을 가져옵니다. 파라미터셋 객체를 반환합니다.'''
    pset_key, description = _action_info.get(action_key, None)
    if not pset_key:
        return None
    return getattr(app.api.HParameterSet, f"H{pset_key}")


In [ ]:
#| export
@patch
def insert_text(app:App, text:str):
    '''`text를 입력합니다.'''
    action = app.actions.InsertText()
    p = action.pset
    p.Text = text
    return action.run()

In [ ]:
app.insert_text("test")

In [ ]:
#| export
@patch
def find_files(app:App):
    '''`text를 입력합니다.'''
    action = app.actions.InsertText
    p = action.pset
    p.Text = text
    return action.run()

In [ ]:
#| export
mask_options = {
    "Normal": 0x00,         # "본문을 대상으로 검색한다.(서브리스트를 검색하지 않는다.)"
    "Char": 0x01,           # "char 타입 컨트롤 마스크를 대상으로 한다.(강제줄나눔, 문단 끝, 하이픈, 묶움빈칸, 고정폭빈칸, 등...)"
    "Inline": 0x02,         # "inline 타입 컨트롤 마스크를 대상으로 한다.(누름틀 필드 끝, 등...)"
    "Ctrl": 0x04,           # "extende 타입 컨트롤 마스크를 대상으로 한다.(바탕쪽, 프레젠테이션, 다단, 누름틀 필드 시작, Shape Object, 머리말, 꼬리말, 각주, 미주, 번호관련 컨트롤, 새 번호 관련 컨트롤, 감추기, 찾아보기, 글자 겹침, 등...)"
    "All": None
}

scan_spos_keys = {
    "Current": 0x0000,      # "캐럿 위치부터. (시작 위치)",
    "Specified": 0x0010,    # "특정 위치부터. (시작 위치)",
    "Line": 0x0020,         # "줄의 시작부터. (시작 위치)",
    "Paragraph": 0x0030,    # "문단의 시작부터. (시작 위치)"
    "Section": 0x0040,      # "구역의 시작부터. (시작 위치)"
    "List": 0x0050,         # "리스트의 시작부터. (시작 위치)"
    "Control": 0x0060,      # "컨트롤의 시작부터. (시작 위치)"
    "Document": 0x0070,     # "문서의 시작부터. (시작 위치)"
}

scan_epos_keys = {
    "Current": 0x0000,      # "캐럿 위치까지. (끝 위치)"
    "Specified": 0x0001,    # "특정 위치까지. (끝 위치)"
    "Line": 0x0002,         # "줄의 끝까지. (끝 위치)"
    "Paragraph": 0x0003,    # "문단의 끝까지. (끝 위치)"
    "Section": 0x0004,      # "구역의 끝까지. (끝 위치)"
    "List": 0x0005,         # "리스트의 끝까지. (끝 위치)"
    "Control": 0x0006,      # "컨트롤의 끝까지. (끝 위치)"
    "Document": 0x0007,     # "문서의 끝까지. (끝 위치)"
}

scan_directions = {
    "Forward": 0x0000, # "정뱡향. (검색 방향)"
    "Backward": 0x0100  # "역방향. (검색 방향)"
}


def _get_text(app):
    """스캔한 텍스트 텍스트 제너레이터"""
    flag, text = 2, ""
    while flag not in [0, 1, 101, 102]:
        flag, text = app.api.GetText()
        yield text

@patch
@contextmanager
def scan(app:App, option="All", selection=False, scan_spos="Document", scan_epos="Document", spara=None, spos=None, epara=None, epos=None, scan_direction="Forward"):
    
    # set start and end position
    spos_id = get_value(scan_spos_keys, scan_spos)
    epos_id = get_value(scan_epos_keys, scan_epos)
    range_ = spos_id + epos_id
    # if selection
    if selection:
        range_ = 0x00ff    # "검색의 범위를 블록으로 제한."
    
    # set direction
    direction = get_value(scan_directions, scan_direction)
    range_ = range_ + direction
    app.api.InitScan(option=get_value(mask_options, option), Range=range_, spara=spara, spos=spos, epara=epara, epos=epos)
    yield _get_text(app)   
    app.api.ReleaseScan()

In [ ]:
#| export
move_ids = {
    "Main": 0,    # 루트 리스트의 특정 위치.(para pos로 위치 지정)
    "CurList": 1,    # 현재 리스트의 특정 위치.(para pos로 위치 지정)
    "TopOfFile": 2,   #문서의 시작으로 이동.
    "BottomOfFile": 3,   # 문서의 끝으로 이동.
    "TopOfList": 4,    # 현재 리스트의 시작으로 이동
    "BottomOfList": 5,   # 현재 리스트의 끝으로 이동
    "StartOfPara": 6,   # 현재 위치한 문단의 시작으로 이동
    "EndOfPara": 7,  # 현재 위치한 문단의 끝으로 이동
    "StartOfWord": 8,  # 현재 위치한 단어의 시작으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "EndOfWord": 9,  # 현재 위치한 단어의 끝으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextPara": 10, # 다음 문단의 시작으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevPara": 11, # 앞 문단의 끝으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextPos": 12, # 한 글자 뒤로 이동.(서브 리스트를 옮겨 다닐 수 있다.)
    "PrevPos": 13, # 한 글자 앞으로 이동.(서브 리스트를 옮겨 다닐 수 있다.)
    "NextPosEx": 14,  # 한 글자 뒤로 이동.(서브 리스트를 옮겨 다닐 수 있다. 머리말/꼬리말, 각주/미주, 글상자 포함.)
    "PrevPosEx": 15,   # 한 글자 앞으로 이동.(서브 리스트를 옮겨 다닐 수 있다. 머리말/꼬리말, 각주/미주, 글상자 포함.)
    "NextChar": 16,  # 한 글자 뒤로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevChar": 17,  # 한 글자 앞으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextWord": 18, # 한 단어 뒤로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevWord": 19, # 한 단어 앞으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextLine": 20, # 한 줄 아래로 이동.
    "PrevLine": 21, # 한 줄 위로 이동.
    "StartOfLine": 22, # 현재 위치한 줄의 시작으로 이동.
    "EndOfLine": 23,  # 현재 위치한 줄의 끝으로 이동.
    "ParentList": 24, # 한 레벨 상위로 이동한다.
    "TopLevelList": 25, # 탑레벨 리스트로 이동한다.
    "RootList": 26, # 루트 리스트로 이동한다. 현재 루트 리스트에 위치해 있어 더 이상 상위 리스트가 없을 때는 위치 이동 없이 반환한다. 이동한 후의 위치는 상위 리스트에서 서브리스트가 속한 컨트롤 코드가 위치한 곳이다. 위치 이동시 셀렉션은 무조건 풀린다.
    "CurrentCaret": 27, # 현재 캐럿이 위치한 곳으로 이동한다. (캐럿 위치가 뷰의 맨 위쪽으로 올라간다. )
    "LeftOfCell": 100,  # 현재 캐럿이 위치한 셀의 왼쪽
    "RightOfCell": 101,  # 현재 캐럿이 위치한 셀의 오른쪽
    "UpOfCell": 102,  # 현재 캐럿이 위치한 셀의 위쪽
    "DownOfCell": 103,  # 현재 캐럿이 위치한 셀의 아래쪽
    "StartOfCell": 104, # 현재 캐럿이 위치한 셀에서 행(row)의 시작
    "EndOfCell": 105, # 현재 캐럿이 위치한 셀에서 행(row)의 끝
    "TopOfCell": 106, # 현재 캐럿이 위치한 셀에서 열(column)의 시작
    "BottomOfCell": 107, #현재 캐럿이 위치한 셀에서 열(column)의 끝
    "ScrPos": 200, # 한/글 문서장에서의 screen 좌표로서 위치를 설정 한다.
    "ScanPos": 201, # GetText() 실행 후 위치로 이동한다.
}

@patch
def move(app:App, move_key="ScanPos", para=None, pos=None):
    """키워드를 바탕으로 캐럿 위치를 이동시킵니다."""
    
    move_id = get_value(move_ids, move_key)
    return app.api.MovePos(moveID=move_id, Para=para, pos=pos)
    

In [ ]:
get_value(move_ids, "ScanPos")

In [ ]:
with app.scan() as scan:
    for text in scan:
        print(text)
        if "청년인턴" in text:
            app.move()

In [ ]:
@patch
def setup_page(app:App, top=20, bottom=10, right=20, left=20, header=15, footer=5, gutter=0):
    """
    페이지를 설정합니다.
    """
    action = app.actions.PageSetup()
    p = action.pset

    p.PageDef.TopMargin = app.api.MiliToHwpUnit(top)
    p.PageDef.HeaderLen = app.api.MiliToHwpUnit(header)
    p.PageDef.RightMargin = app.api.MiliToHwpUnit(right)
    p.PageDef.BottomMargin = app.api.MiliToHwpUnit(bottom)
    p.PageDef.FooterLen = app.api.MiliToHwpUnit(footer)
    p.PageDef.LeftMargin = app.api.MiliToHwpUnit(left)
    p.PageDef.GutterLen = app.api.MiliToHwpUnit(gutter)
    
    return action.run()

In [ ]:
#| export
size_options = {
    "realSize": 0,   # 이미지를 원래의 크기로 삽입한다.
    "specificSize": 1,    # width와 height에 지정한 크기로 그림을 삽입한다.
    "cellSize": 2,    # 현재 캐럿이 표의 셀안에 있을 경우, 셀의 크기에 맞게 자동 조정하여 삽입한다. 
    "cellSizeWithSameRatio":3    # 현재 캐럿이 표의 셀 안에 있을 경우, 셀의 크기에 맞추어 원본 이미지의 가로 세로 비율이 동일하게 확대/축소하여 삽입한다.
}

effects = {
    "RealPicture": 0,   # 원본
    "GrayScale": 1,    # 그레이 스케일
    "BlackWhite": 2,     # 흑백효과
}

@patch
def insert_picture(app:App, fpath, width=None, height=None, size_option="realSize", reverse=False, watermark=False, effect="RealPicture"):
    """
    사이즈를 지정하여 사진 삽입
    """
    path = Path(fpath)
    sizeoption = get_value(size_options, size_option)
    effect = get_value(effects, effect)
    return app.api.InsertPicture(
        path.absolute().as_posix(), 
        Width=width, 
        Height=height, 
        sizeoption=sizeoption,
        reverse=reverse,
        watermark=watermark,
        effect=effect
    )

In [ ]:
#| export
@patch
def get_selected_text(app:App):
    """
    선택된 영역의 텍스트를 불러온다.
    """
    with app.scan(selection=True) as scan:
        text = "\n".join(scan)
    return text


In [ ]:
#| export
directions = {
    "Forward": 0,
    "Backward": 1, 
    "All": 2
}

@patch
def replace_all(app:App, 
                old = "",
                new = "",
                old_fontcolor=None,  # 찾을 폰트 색
                new_fontcolor=None,  # 바꿀 폰트 색
                old_fontsize=None,   # 찾을 폰트 크기(height)
                new_fontsize=None,   # 바꿀 폰트 크기(height)
                old_fontname="",  # 찾을 글꼴
                old_fonttype=1,   # 찾을 글꼴 타입 TTF = 1, HTF = 2
                new_fontname="",  # 바꿀 글꼴 
                new_fonttype=1,   # 바꿀 글꼴 타입 TTF = 1, HTF = 2
                old_fontratio=None,  # 찾을 장평
                new_fontratio=None,  # 바꿀 장평
                old_spacing=None,  # 찾을 자간
                new_spacing=None,  # 바꿀 자간
                old_bold=None,  # 찾을 볼드
                new_bold=None,  # 바꿀 볼드
                old_italic=None, # 찾을 이텔릭
                new_italic=None,  # 바꿀 이텔릭
                old_underline=None,  # 찾을 밑줄
                new_underline=None,  # 바꿀 밑줄
                old_strike_out=None,  # 찾을 취소선
                new_strike_out=None,  # 바꿀 취소선
                ignore_message=True,  # 메시지 무시 여부
                direction="All",   # 찾을 방향
                match_case=False,  # 글자 
                all_word_forms=False,  # 
                several_words=False,
                use_wild_cards=False,
                whole_word_only=False,
                auto_spell=True,
                replace_mode=True,
                ignore_find_string=False,
                ignore_replace_string=False,
                find_style="",
                replace_style="",
                find_jaso=False,
                find_reg_exp=False,
                find_type=True,
               ):
    
    action = app.actions.AllReplace()
    p = action.pset
    
    # set options
    p.FindString = old
    p.ReplaceString = new
    p.IgnoreMessage = ignore_message
    p.MatchCase = match_case
    p.AllWordForms = all_word_forms
    p.Direction = get_value(directions, direction)
    p.SeveralWords = several_words
    p.UseWildCards = use_wild_cards
    p.WholeWordOnly = whole_word_only
    p.AutoSpell = auto_spell
    p.ReplaceMode = replace_mode
    p.IgnoreFindString = ignore_find_string
    p.IgnoreReplaceString = ignore_replace_string
    p.FindStyle = find_style
    p.ReplaceStyle = replace_style
    p.FindJaso = find_jaso
    p.FindRegExp = find_reg_exp
    p.FindType = find_type

    # set old charshape
    olds = []
    categories = ["Hangul", "Latin", "Hanja", "Japanese", "Other", "User"]
    
    olds += [("FaceName"+cat, old_fontname) for cat in categories] if old_fontname else []
    olds += [("FontType"+cat, old_fonttype) for cat in categories] if old_fontname else []
    olds += list(
        filter(lambda x: x[1], 
            [
                ("TextColor", old_fontcolor),
                ("Height", app.api.PointToHwpUnit(old_fontsize) if old_fontsize else None),
                ("Bold", old_bold),
                ("Italic", old_italic),
                ("StrikeOutType", old_strike_out),
                ("UnderlineType", old_underline),
                ("RatioHangul", old_fontratio),
                ("RatioLatin", old_fontratio),
                ("RatioHanja", old_fontratio),
                ("RatioJapanese", old_fontratio),
                ("RatioOther", old_fontratio),
                ("RatioUser", old_fontratio),
            ]
        )
    ) 
    for key, value in olds:
            setattr(p.FindCharShape, key, value)

            
    # set new charshape
    news = []
    news += [("FaceName"+cat, new_fontname) for cat in categories] if new_fontname else []
    news += [("FontType"+cat, new_fonttype) for cat in categories] if new_fontname else []
    news += list(
        filter(lambda x: x[1], 
            [
                ("TextColor", new_fontcolor),
                ("Height", app.api.PointToHwpUnit(new_fontsize) if new_fontsize else None),
                ("Bold", new_bold),
                ("Italic", new_italic),
                ("StrikeOutType", new_strike_out),
                ("UnderlineType", new_underline),
                ("RatioHangul", new_fontratio),
                ("RatioLatin", new_fontratio),
                ("RatioHanja", new_fontratio),
                ("RatioJapanese", new_fontratio),
                ("RatioOther", new_fontratio),
                ("RatioUser", new_fontratio),
            ]
        )
    )
    
    for key, value in news:
        setattr(p.ReplaceCharShape, key, value)
        
    return action.run()
    

In [ ]:
app.replace_all("t", "txt", new_fontname="가는안상수체", new_fontsize=21, new_fonttype=1)

In [ ]:
app.replace_all("", "", old_fontname="바탕체", new_fontname="바탕체", ignore_find_string=True, ignore_replace_string=True, new_fontsize=11, new_fonttype=1)

In [ ]:
#| export
@patch
def insert_file(app:App, fpath, keep_charshape=False, keep_parashape=False, keep_section=False, keep_style=False):
    """
    파일 끼워 넣기
    """
    
    action = app.actions.InsertFile()
    p = action.pset
    p.filename= Path(fpath).absolute().as_posix()
    p.KeepCharshape = keep_charshape
    p.KeepParashape = keep_parashape
    p.KeepSection = keep_section
    p.KeepStyle = keep_style
    
    return action.run()

한글 문서 사용은 다음과 같이 할 수 있습니다. 
먼저 App 객체를 생성합니다.

In [ ]:
app.get_selected_text()

In [ ]:
p = app.actions.InsertFile().pset

In [ ]:
dir(p)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()